<a href="https://colab.research.google.com/github/Chella2049/Chellapandi/blob/main/Assign_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gradio as gr
from mlxtend.frequent_patterns import apriori, association_rules

# Function to process CSV and apply Apriori
def generate_association_rules(file, min_support, min_confidence):
    df = pd.read_csv(file.name)

    # Convert categorical data into transaction format
    df = df.applymap(lambda x: str(x).strip())  # Ensure proper formatting
    transactions = df.values.tolist()
    unique_items = sorted(set(item for sublist in transactions for item in sublist if item))

    # Create one-hot encoded DataFrame
    encoded_data = pd.DataFrame([[1 if item in row else 0 for item in unique_items] for row in transactions], columns=unique_items)

    # Apply Apriori Algorithm
    frequent_itemsets = apriori(encoded_data, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

    return rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_string(index=False)

# Gradio Interface
demo = gr.Interface(
    fn=generate_association_rules,
    inputs=[gr.File(), gr.Slider(0.01, 1.0, step=0.01, label="Min Support"), gr.Slider(0.1, 1.0, step=0.05, label="Min Confidence")],
    outputs=gr.Textbox(label="Association Rules"),
    title="Apriori Algorithm - Association Rule Mining",
    description="Upload a CSV file containing transaction data. The Apriori algorithm will generate association rules based on the chosen parameters."
)

demo.launch()

/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://743bac11010706ab54.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.cluster import KMeans
import gradio as gr

# Load sample movie ratings dataset (assumed to be in user-item matrix format)
def load_sample_data():
    data = {
        'User1': [5, 4, 0, 1, 0],
        'User2': [4, 5, 1, 0, 0],
        'User3': [0, 0, 5, 4, 3],
        'User4': [0, 1, 4, 5, 2],
        'User5': [3, 3, 4, 5, 5],
    }
    df = pd.DataFrame(data, index=['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E'])
    return df.T

ratings = load_sample_data()
n_users, n_movies = ratings.shape

# Autoencoder model
input_layer = Input(shape=(n_movies,))
encoded = Dense(4, activation='relu')(input_layer)
encoded = Dense(2, activation='relu')(encoded)  # Latent space (compressed representation)
decoded = Dense(4, activation='relu')(encoded)
decoded = Dense(n_movies, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train autoencoder
X_train = ratings.fillna(0).values  # Replace NaN with 0
autoencoder.fit(X_train, X_train, epochs=100, batch_size=2, verbose=0)

# Extract user embeddings
encoder = Model(input_layer, encoded)
user_embeddings = encoder.predict(X_train)

# Cluster users using K-Means
n_clusters = 2  # Define number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(user_embeddings)
user_clusters = kmeans.labels_

# Function to recommend movies based on user clustering
def recommend_movies(user_ratings):
    user_ratings = np.array([float(x) for x in user_ratings.split(',')])
    user_embedding = encoder.predict(user_ratings.reshape(1, -1))
    cluster = kmeans.predict(user_embedding)[0]
    cluster_users = ratings.iloc[np.where(user_clusters == cluster)]
    avg_ratings = cluster_users.mean().sort_values(ascending=False)
    return avg_ratings.index.tolist()

# Gradio Interface
demo = gr.Interface(
    fn=recommend_movies,
    inputs=gr.Textbox(placeholder="Enter ratings as comma-separated values, e.g., 5,4,0,1,0"),
    outputs=gr.Textbox(label="Recommended Movies"),
    title="Deep Learning Movie Recommender",
    description="Enter your movie ratings (comma-separated). The system will cluster you with similar users and recommend movies accordingly."
)

demo.launch()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://166f9f9c11378266ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.cluster import KMeans
import gradio as gr

# Download VADER sentiment analyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Sample dataset: Social media comments
data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7'],
    'Comment': [
        "I love this product! Absolutely amazing!",  # Positive
        "Worst experience ever. Never buying again!",  # Negative
        "It's okay, nothing special but not bad either.",  # Neutral
        "Fantastic service, very satisfied!",  # Positive
        "I hate it, total waste of money!",  # Negative
        "Mediocre at best, but decent value.",  # Neutral
        "Superb quality, exceeded expectations!"  # Positive
    ]
}

df = pd.DataFrame(data)

# Sentiment Analysis Function
def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    return [scores['neg'], scores['neu'], scores['pos']]

# Convert comments to sentiment scores
df[['Negative', 'Neutral', 'Positive']] = df['Comment'].apply(analyze_sentiment).apply(pd.Series)

# Apply K-Means clustering
n_clusters = 3  # Positive, Neutral, Negative
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(df[['Negative', 'Neutral', 'Positive']])

# Function to predict sentiment cluster for new comments
def predict_cluster(comment):
    sentiment_scores = np.array(analyze_sentiment(comment)).reshape(1, -1)
    cluster = kmeans.predict(sentiment_scores)[0]
    cluster_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    return f"Predicted Sentiment: {cluster_labels[cluster]}"

# Gradio Interface
demo = gr.Interface(
    fn=predict_cluster,
    inputs=gr.Textbox(placeholder="Enter a comment..."),
    outputs=gr.Textbox(label="Sentiment Cluster"),
    title="Sentiment-Based Clustering for Opinion Analysis",
    description="Enter a social media comment, and the system will analyze its sentiment and categorize it into a cluster (Positive, Neutral, Negative)."
)

demo.launch()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://655bccab988fcf9208.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans
import gradio as gr

# Sample Crime Dataset (Latitude, Longitude, Crime Type)
data = {
    'Latitude': [37.7749, 37.7750, 37.7755, 37.7800, 37.7700, 37.7790, 37.7760],
    'Longitude': [-122.4194, -122.4185, -122.4175, -122.4160, -122.4205, -122.4150, -122.4140],
    'Crime Type': ['Theft', 'Assault', 'Burglary', 'Robbery', 'Theft', 'Vandalism', 'Assault']
}

df = pd.DataFrame(data)

# Function to perform K-Means clustering and generate hotspot map
def generate_crime_map(n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

    # Create Map
    crime_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=13)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred']  # Colors for clusters

    # Add clustered crime locations to the map
    marker_cluster = MarkerCluster().add_to(crime_map)
    for i, row in df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Crime Type']} - Cluster {row['Cluster']}",
            icon=folium.Icon(color=colors[row['Cluster'] % len(colors)])
        ).add_to(marker_cluster)

    return crime_map._repr_html_()

# Gradio Interface
demo = gr.Interface(
    fn=generate_crime_map,
    inputs=gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Number of Clusters"),
    outputs=gr.HTML(label="Crime Hotspot Map"),
    title="Crime Hotspot Detection Using K-Means",
    description="Adjust the number of clusters to analyze crime hotspots."
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52db71a3c92fede701.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
